In [2]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import plotly.express as px
import plotly.io as pio
from scipy import stats
pio.renderers.default = "vscode"
pio.renderers
from plotly.graph_objs import *
from song_characteristics import *
from plots import *
from spot_secrets import *
import requests
import urllib.request

In [3]:
user_id = "93f1918c-df6a-4c46-94e9-11d35c979f03"

In [4]:
conn, cursor = db_connect("spotify_db.db")
sp = connect_to_sp(cid, secret)
dataframe = download_data(user_id, cursor)

In [ ]:
counts = dataframe.groupby("song_name")["song_name"].size().reset_index(name="count")

In [ ]:
dataframe["sec_played"] = dataframe["ms_played"]/1000

In [ ]:
dataframe = dataframe.drop(["ms_played"], axis=1)
dataframe = dataframe.groupby("song_name")["song_name"].size().reset_index(name="count")
dataframe = dataframe.groupby('count').size().reset_index(name='number_of_songs')
plot_df = dataframe.copy()
fig = px.scatter(x=plot_df["number_of_songs"], y=plot_df["count"], labels={"x":"Number of songs", "y":"Number of plays"}, color_discrete_sequence=['#1DB954'])
fig.update_layout(
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    title_font_color="white",
    title = {
        "text":"Do you attach to songs?",
        "x": 0.5,
        "xanchor":"center",
        "yanchor": "top"
    }
)
fig.update_xaxes(color="white")
fig.update_yaxes(color="white")

In [ ]:
dataframe = download_data(user_id, cursor)

In [ ]:
total_listening_time, favorite_artists, favorite_songs, distinct_artists, distinct_songs, favorite_artists_minutes, favorite_artist_fraction, favorite_songs_of_fav_artist, number_of_songs_by_fav_artist, favorite_morning, favorite_evening = get_general_statistics(dataframe)

In [ ]:
favorite_songs_of_fav_artist

In [ ]:
favorite_artists

In [ ]:
dataframe["min_played"] = dataframe["ms_played"]/60000

In [ ]:
total_listening_time = dataframe["min_played"].sum() # Total listening time
favorite_artists = dataframe.groupby("artist_name")["artist_name"].size().reset_index(name="count").sort_values("count", ascending=False) # Favorite artists
favorite_songs = dataframe.groupby(["artist_name", "song_name"]).size().reset_index(name="count").sort_values("count", ascending=False) # Favorite songs
distinct_artists = len(dataframe.drop_duplicates("artist_name")) # Distinct artists
distinct_songs = len(dataframe.drop_duplicates("song_name")) # Distinct songs


In [ ]:
# Stats for favorite artist
favorite_artists_minutes = dataframe.groupby("artist_name")["min_played"].sum().astype(int).reset_index(name="sum_of_minutes").sort_values("sum_of_minutes", ascending=False) # Minutes of favorite artists
favorite_artist_fraction = favorite_artists_minutes["sum_of_minutes"].iloc[0] / favorite_artists_minutes["sum_of_minutes"][1:].sum() # Percentage of minutes occupied by favorite artist
favorite_songs_of_fav_artist = favorite_songs[favorite_songs["artist_name"] == favorite_artists["artist_name"].iloc[0]] # Favorite songs of favorite artist
number_of_songs_by_fav_artist = len(favorite_songs_of_fav_artist) # Number of distinct songs of favorite artist listened

In [ ]:
dataframe["hour"] = pd.to_datetime(dataframe["end_time"]).dt.hour
dataframe_evening = dataframe[(dataframe["hour"].astype(int) > 19) | (dataframe["hour"].astype(int) < 5)]
dataframe_morning = dataframe[(dataframe["hour"].astype(int) > 5) & (dataframe["hour"].astype(int) < 10)]

In [ ]:
favorite_evening = dataframe_evening.groupby(["artist_name", "song_name"]).size().reset_index(name="count").sort_values("count", ascending=False) # Favorite morning songs
favorite_morning = dataframe_morning.groupby(["artist_name", "song_name"]).size().reset_index(name="count").sort_values("count", ascending=False) # Favorite evening songs

In [ ]:
favorite_morning.head()

In [ ]:
dataframe['date'] = pd.to_datetime(dataframe['end_time'])

In [ ]:
dataframe['date'] = pd.to_datetime(dataframe['date'].dt.date)

In [ ]:
dataframe['hour'] = pd.to_datetime(dataframe['end_time']).dt.hour

In [ ]:
dataframe['week'] = pd.to_datetime(dataframe['end_time']).dt.week

In [ ]:
df_grouped = dataframe.groupby(['week', 'hour']).size().reset_index(name='count')

In [ ]:
dataframe[(dataframe["week"] == 48) & (dataframe["hour"] == 0)]

In [ ]:
df_grouped[df_grouped["week"] == 48]

In [ ]:
make_general_heatmap(dataframe)

In [54]:
personal_data = get_data_with_statistics(user_id, cursor)
recommender_data = get_data_from_recommender(cursor)

In [5]:
personal_data.head()

,artist_name,song_name,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,end_time
0,Playboi Carti,Shoota (feat. Lil Uzi Vert),0.674,0.617,-9.680,0.2220,0.1850,0.000,0.1060,0.451,153.103,2022-05-14 09:18
1,Playboi Carti,Fell In Luv (feat. Bryson Tiller),0.657,0.668,-6.208,0.1360,0.0273,0.000,0.3200,0.227,162.997,2022-05-14 09:18
2,Playboi Carti,Magnolia,0.791,0.582,-7.323,0.2860,0.0114,0.000,0.3500,0.443,162.991,2022-05-14 09:21
3,Trippie Redd,Miss The Rage (feat. Playboi Carti),0.581,0.877,-4.938,0.1140,0.0151,0.000,0.8260,0.353,77.006,2022-05-14 17:34
4,070 Shake,Terminal B,0.703,0.393,-11.191,0.0374,0.0755,0.819,0.0876,0.153,135.009,2022-05-14 17:40


In [55]:
recommender_results = recommend_me(personal_data, recommender_data, ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"])

0       0.743905
1       0.850776
2       0.816455
3       0.889867
4       0.697396
          ...   
1298    0.788476
1299    0.735071
1300    0.751231
1301    0.567810
1302    0.559684
Name: loudness, Length: 1303, dtype: float64


In [57]:
recommender_results

{'artist_personal': ['Miły ATZ',
  'Floating Points',
  'Airelle Besson',
  'Charli XCX',
  'Leon Vynehall'],
 'song_personal': ['Day Off',
  'Movement 5',
  'The Sound of Your Voice, Pt. II',
  'Selfish Girl',
  'Movements (Chapter III)'],
 'artist_database': ['jonatan leandoer96',
  'Sleep Party People',
  'Ross from Friends',
  'The Temper Trap',
  'R.A.P. Ferreira'],
 'song_database': ['Vår Sång',
  'Everything Has an End',
  'A Brand New Start',
  'Sweet Disposition',
  'CYCLES'],
 'distance': [1.0255009115625726,
  0.44983759629795655,
  0.42817657592525227,
  0.3884818924487179,
  0.3525853953993009]}

In [58]:
def get_artists_photos(results_dict, sp):
    artist_ids = []

    for i in range(len(results_dict)):
        artist_personal_name = results_dict["artist_personal"][i]
        artist_recommender_name = results_dict["artist_database"][i]
        result_personal = sp.search(q=artist_personal_name, type='artist')
        artist_id_personal = result_personal['artists']['items'][0]['id']
        result_recommender = sp.search(q=artist_recommender_name, type='artist')
        artist_id_recommender = result_recommender['artists']['items'][0]['id']
        artist_search_personal = sp.artist(artist_id_personal)
        artist_search_recommender = sp.artist(artist_id_recommender)
        artist_image_url_personal = artist_search_personal['images'][0]['url']
        artist_image_url_recommender = artist_search_recommender['images'][0]['url']

        urllib.request.urlretrieve(artist_image_url_personal, "../website/static/artist_image_personal" + str(i) + ".jpg")
        urllib.request.urlretrieve(artist_image_url_recommender, "../website/static/artist_image_recommender" + str(i) + ".jpg")


    

In [59]:
get_artists_photos(recommender_results, sp)

In [5]:
data = get_song_stats_by_date_with(user_id, cursor)

In [6]:
data.head()

,artist_name,song_name,danceability,energy,tempo,acousticness,loudness,instrumentalness,end_time
0,Męskie Granie Orkiestra,Elektryczny,0.652,0.816,139.931,0.19700,-4.299,0.000036,2022-01-01 01:52
1,Muniek Staszczyk,Pola,0.600,0.732,170.024,0.00924,-7.472,0.000005,2022-01-01 20:59
2,Lorde,Stoned at the Nail Salon,0.473,0.128,93.950,0.94800,-12.303,0.000054,2022-01-01 20:59
3,Kings of Leon,Pyro,0.365,0.606,114.995,0.00757,-7.174,0.052000,2022-01-01 21:03
4,Imagine Dragons,I’m So Sorry,0.528,0.687,142.605,0.14400,-5.954,0.000000,2022-01-01 21:07


In [66]:
data["end_time"]

0      2022-01-01 01:52
1      2022-01-01 20:59
2      2022-01-01 20:59
3      2022-01-01 21:03
4      2022-01-01 21:07
             ...       
852    2022-01-27 21:53
853    2022-01-27 21:58
854    2022-01-27 22:01
855    2022-01-27 22:05
856    2022-01-28 11:28
Name: end_time, Length: 857, dtype: object

In [15]:
data["month"] = pd.to_datetime(data["end_time"]).dt.month

In [62]:
data.shape()

TypeError: 'tuple' object is not callable

In [59]:
data = data.groupby("month").agg("mean").reset_index()

In [60]:
data["month"].value_counts()

1    1
Name: month, dtype: int64

In [47]:
def song_statistics_through_the_year(dataframe):
    dataframe = normalize(dataframe, ["tempo", "loudness"])
    dataframe["month"] = pd.to_datetime(dataframe["end_time"]).dt.month
    plot_data = dataframe.groupby("month").agg("mean").reset_index()
    plot_data.head()
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=plot_data['danceability'], x=plot_data['month'],
                        mode='lines',
                        name='danceability'))
    fig.add_trace(go.Scatter(y=plot_data['energy'], x=plot_data['month'],
                             mode='lines',
                             name='energy'))
    fig.add_trace(go.Scatter(y=plot_data['tempo'], x=plot_data['month'],
                             mode='lines',
                             name='tempo'))
    fig.add_trace(go.Scatter(y=plot_data['acousticness'], x=plot_data['month'],
                             mode='lines',
                             name='acousticnesss'))
    fig.add_trace(go.Scatter(y=plot_data['instrumentalness'], x=plot_data['month'],
                             mode='lines',
                             name='instrumentalness'))
    fig.show()
    return fig


In [56]:
data= normalize(data, ["tempo", "loudness"])

In [57]:
data.head()

,danceability,energy,tempo,acousticness,loudness,instrumentalness,end_time
0,0.652,0.816,0.638561,0.19700,0.921648,0.000036,2022-01-01 01:52
1,0.600,0.732,0.775887,0.00924,0.847789,0.000005,2022-01-01 20:59
2,0.473,0.128,0.428731,0.94800,0.735335,0.000054,2022-01-01 20:59
3,0.365,0.606,0.524768,0.00757,0.854725,0.052000,2022-01-01 21:03
4,0.528,0.687,0.650763,0.14400,0.883124,0.000000,2022-01-01 21:07


In [85]:
data.agg("mean")["loudness"]

0.7966068783015884

In [54]:
song_statistics_through_the_year(data)

KeyError: 'end_time'

In [29]:
data = data.agg("mean")

In [43]:
data["tempo"]

119.49083471259235

In [100]:
def make_radar(dataframe):
    dataframe = normalize(dataframe, ["tempo", "loudness"])
    dataframe = dataframe.agg("mean")

    #fig.add_trace\
    fig = go.Figure(go.Scatterpolar(
        name = "Your songs with highest:",
        r = [dataframe["danceability"], dataframe["energy"], dataframe["acousticness"], dataframe["instrumentalness"], dataframe["loudness"], dataframe["tempo"]],
        theta = ["danceability", "energy", "accousticness", "instrumentalness", "loudness", "tempo"],
        line_color="green",
        fill="toself"
    ))
    fig.update_layout(
        polar = dict(
        bgcolor="rgb(223, 223, 223)",
        angularaxis = dict(
        direction = "clockwise",
        period = 6)),
        title = {
                "text":"What are the specifics of the songs you listen to?",
                "x": 0.5,
                "xanchor":"center",
                "yanchor": "top"}, 
        width=1600,
        height=800
    )
    return fig

In [101]:
fig = make_radar(data)

In [102]:
fig.show()

In [7]:
data = get_song_stats_by_date_with_names(user_id, cursor)

In [8]:
data.head()

,artist_name,song_name,danceability,energy,tempo,acousticness,loudness,instrumentalness,end_time
0,Męskie Granie Orkiestra,Elektryczny,0.652,0.816,139.931,0.19700,-4.299,0.000036,2022-01-01 01:52
1,Muniek Staszczyk,Pola,0.600,0.732,170.024,0.00924,-7.472,0.000005,2022-01-01 20:59
2,Lorde,Stoned at the Nail Salon,0.473,0.128,93.950,0.94800,-12.303,0.000054,2022-01-01 20:59
3,Kings of Leon,Pyro,0.365,0.606,114.995,0.00757,-7.174,0.052000,2022-01-01 21:03
4,Imagine Dragons,I’m So Sorry,0.528,0.687,142.605,0.14400,-5.954,0.000000,2022-01-01 21:07


In [15]:
max_danceability = data[data["danceability"] == np.max(data["danceability"])].iloc[0]
max_energy = data[data["energy"] == np.max(data["energy"])].iloc[0]
max_tempo = data[data["tempo"] == np.max(data["tempo"])].iloc[0]
max_loudness = data[data["loudness"] == np.max(data["loudness"])].iloc[0]
max_instrumental = data[data["instrumentalness"] == np.max(data["instrumentalness"])].iloc[0]
max_acoustic = data[data["acousticness"] == np.max(data["acousticness"])].iloc[0]

In [21]:
maxes_df = pd.DataFrame([max_danceability, max_energy, max_tempo, max_loudness, max_instrumental, max_acoustic], columns = ["artist_name", "song_name", "danceability", "energy", "tempo", "acousticness", "loudness", "instrumentalness", "end_time"])

,artist_name,song_name,danceability,energy,tempo,acousticness,loudness,instrumentalness,end_time
3632,Theodore Shapiro,Problems with Women,0.970,0.1700,116.021,0.6480,-16.974,0.8650,2022-09-03 12:41
1896,Foster The People,Miss You,0.625,0.9950,120.991,0.0219,-3.950,0.0307,2022-03-25 15:43
4455,Beach House,Lazuli,0.341,0.8100,219.135,0.0105,-5.790,0.6290,2022-12-06 18:04
2277,Amyl and The Sniffers,No More Tears,0.224,0.9820,174.533,0.0909,-0.933,0.0000,2022-04-06 09:47
3432,Nina Nastasia,Creek and Chimes,0.180,0.3180,107.366,0.7400,-38.366,0.9990,2022-08-15 20:09
3446,Florist,Variation,0.319,0.0254,70.890,0.9960,-23.477,0.0262,2022-08-15 20:58
